In [2]:
!pip install minsearch


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import minsearch
import json
import requests
import os

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
q = 'the course has already started, can I still enroll?'

In [9]:
index.fit(documents)

In [10]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=q,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [11]:
api_key = os.getenv("GROQ_API_KEY")

In [12]:
url = "https://api.groq.com/openai/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

In [33]:
data = {
    "model": "llama3-70b-8192",  
    "messages": [
        {"role": "user", "content": prompt}
    ]
}

response = requests.post(url, headers=headers, json=data)

resp_json = response.json()
resp_json["choices"][0]["message"]["content"]

'According to the FAQ database, the answer to the question "Can I still enroll in the course now that it has already started?" is:\n\nYes, even if you don\'t register, you\'re still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don\'t leave everything for the last minute.'

First you you have query (or question), 
Then, you do a search to get related documents, producing search_results. 
The query and search_results are used to build a prompt, (with new context, to augment data in LLM). 
The prompt is then fed into the llm which then returns and answer.

In [34]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [36]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [41]:
def llm(prompt):
    data = {
        "model": "llama3-70b-8192",  
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }

    response = requests.post(url, headers=headers, json=data)

    resp_json = response.json()
    return resp_json["choices"][0]["message"]["content"]

In [44]:
query = 'how do I run kafka?'

def rag(query):
    """
    RAG function to search for relevant documents and generate an answer.
    """
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [48]:
rag(query)

'Based on the context provided, there is no direct question or answer that explains how to run Kafka. However, there are several questions and answers related to running Kafka-related code, such as running a Java Kafka producer, running a Python Kafka producer, and resolving issues with running Kafka code.\n\nTo run Kafka, you might need to:\n\n1. Create a virtual environment and run the Python files in that environment.\n3. Install necessary dependencies, such as `dlt[duckdb` or `kafka-python-ng`.\n4. Run a script, like `build.sh`, in a specific directory with the correct permissions.\n\nPlease provide more context or clarify what you mean by "run Kafka" for a more accurate answer.'